In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

catalog_name='ecommerce'

### Brands Table

In [0]:
df_bronze = spark.table(f"{catalog_name}.bronze.brz_brands")
df_bronze.show(10)

In [0]:
df_silver=df_bronze.withColumn('brand_name', F.trim(F.col('brand_name')))
df_silver.show(20)

In [0]:
df_silver = df_silver.withColumn("brand_code", F.regexp_replace(F.col("brand_code"),r'[^A-Za-z0-0]',''))
df_silver.show(20)

In [0]:
df_silver.select("category_code").distinct().show()

In [0]:
#Anamolies
anamolies = {
    "GROCERY" : "GRCY",
    "BOOKS" : "BKS" , 
    "TOYS" : "TOY"
}

df_silver = df_silver.replace(anamolies, subset="category_code")
df_silver.select("category_code").distinct().show()

In [0]:
df_silver.write.format("delta")\
  .option("overwriteSchema","true")\
    .mode("overwrite")\
      .saveAsTable(f"{catalog_name}.silver.slv_brands")

### Category Table

In [0]:
df_bronze = spark.table(f"{catalog_name}.bronze.brz_category")
display(df_bronze.limit(10))

In [0]:
df_duplicate = df_bronze.groupBy("category_code").count().filter(F.col("count")>1).show()

In [0]:
df_silver = df_bronze.dropDuplicates(['category_code'])
df_silver = df_bronze.withColumn("category_code", F.upper(F.col("category_code")))
display(df_silver)

In [0]:
df_silver.write.format("delta")\
    .option("mergeSchema","true") \
        .mode("overwrite")\
            .saveAsTable(f"{catalog_name}.silver.slv_category")